In [1]:
# Volume fluxes for the different layers
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import math

In [2]:
sd = 1 # sill depth

file0 = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/1_BaseCase/state_' + str(format(sd,'03d')) + '.nc')
grid = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/1_BaseCase/grid_' + str(format(sd,'03d')) + '.nc')

file = file0.isel(T=~file0.get_index("T").duplicated())
state1 = file.isel(X=range(260), Xp1=range(261), Y=range(35,45), Yp1=range(35,45), T=range(90,121))

HFacW = grid.HFacW.data[:,35:45,:] # vertical fraction of open cell at West face
rA = grid.rA.data[35:45,:] # r face area at U point


# Area = np.empty([50, 9]) # Cross Y direction
# Area[:10, :] = 100
# Area[10:20, :] = 200
# Area[20:30, :] = 300
# Area[30:40, :] = 800
# Area[40:, :] = 1600

Area = np.empty([90, 10]) # Cross Y direction
Area[:20, :] = 400
Area[20:50, :] = 800
Area[50:, :] = 1200


rho = 999.8 # Density
f = 6
s = 235


In [3]:
# Read data in T,Z,Y,X
u = state1.U.data
#u = 0.5 * (u0[:,:,:,:-1] + u0[:,:,:,1:])
uzx = u.mean(axis=(0,2)) # Z, X
w = state1.W.data
#w = 0.5 * (w0[:,:-1,:,:] + w0[:,1:,:,:])

In [4]:
# Advective flux of potential temperature
Vf = np.sum(Area * u[:,:,:,f].mean(0))
Vs = np.sum(Area * u[:,:,:,s].mean(0))


In [5]:
# Find the interface and vertical flux of theta
def Layer(uzx):
    # Layer interface, zero velocity crossing point
    
    wt = w.mean(0) # Z, Y, X
    X = uzx.shape[1]
    Vzx = np.empty(X)
    itf = np.empty(X, dtype=int)
    Vzx[[0,1,-2,-1]] = 0
    itf[[0,1,-2,-1]] = 0
    for i in range(2,X-2):
        l = np.where(uzx[:, i] < 0)[-1][0]
        itf[i] = l
        if l == 0:
            itf[i] = itf[i-1]
        Vzx[i] = np.sum(wt[l,:,i] * rA[:,i]) # Vertical advective flux through the interface
    return (itf, Vzx)

In [6]:
itf, Vzx = Layer(uzx)

#f = 5 # Control volume start index near the ice front 3
#s = 88 # Control volume end index at the sill crest 28

Vz = Vzx[f:s].sum() 

Vf1 = np.sum(HFacW[:itf[f],:,f] * Area[:itf[f],:] * u[:,:itf[f],:,f].mean(0)) 
Vs1 = np.sum(HFacW[:itf[s],:,s] * Area[:itf[s],:] * u[:,:itf[s],:,s].mean(0)) 

Vf2 = np.sum(HFacW[itf[f]:,:,f] * Area[itf[f]:,:] * u[:,itf[f]:,:,f].mean(0))
Vs2 = np.sum(HFacW[itf[s]:,:,s] * Area[itf[s]:,:] * u[:,itf[s]:,:,s].mean(0))

Vf1, Vs1, Vf2, Vs2, itf

(7153.958104830235,
 2447.4952074699104,
 -6903.955614617735,
 -2198.8531113602226,
 array([ 0,  0, 22, 21, 21, 21, 21, 20, 20, 20, 20, 20, 20, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 19,
        19, 18, 18, 18, 17, 17, 17, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 15, 15, 14, 13, 12, 11, 10, 10,  9,  9,  8,  7,  7,  7,
         6,  6,  6,  6,  5,  5,  5,  5,  5,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,
    

In [7]:
Vzx[f:s].sum()

-5647.461578074766